In [1]:
import geopandas as gpd
from folium import (
    LayerControl,
    Element,
    TileLayer,
)
import plotly.express as px
from os import path, makedirs



from core.downloads.geosampa import get_capabilities, get_features

# Promoção da Sustentabilidade Ambiental, Gestão de risco

## Formulário 7

O formulário associado a este notebook solicita os dados sobre 
`áreas de risco (hidrológico e deslizamentos)`. A justificativa para estes dados é a seguinte:

> As áreas de risco estão relacionadas à construção de moradias, em sua maioria em condições precárias, em locais com geológico-geotécnicas frágeis, não recomendadas para ocupação. O impacto de chuvas concentradas intensas, características de eventos climáticos extremos deve ser monitorado pelo Município, através de políticas públicas de gestão de risco e promoção da resiliência climática.

## Carregando as camadas de risco hidrológico e geológico

In [ ]:
get_capabilities('hidrológico')

In [ ]:
get_capabilities('deslizamento')

Como o formulário cita apenas a camada `proteção e defesa civil/mapeamento de areas de risco`, assumirei que a referência a deslizamentos seja sobre a camada de risco geológico.

In [ ]:
df_hid = get_features('geoportal:risco_hidrologico')
df_hid.head()

In [ ]:
df_geo = get_features('geoportal:area_risco_geologico')
df_geo.head()

Além dos dados de riscos hidrogeológicos, também precisaremos dos dados do Censo de 2022 para a estimativa populacional. Os dados básicos, como número de domicílios e população, são disponibilizados diretamente no geopackage com as geometrias de setores censitários.

In [ ]:
df_censo = gpd.read_file('https://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/setores/gpkg/UF/SP/SP_setores_CD2022.gpkg')
df_censo.head()

In [ ]:
df_censo = df_censo.loc[df_censo['CD_MUN']=='3550308']
df_censo.head()

## Carregando a camada de cobertura vegetal

In [ ]:
get_capabilities('veg')

In [ ]:
df_veg = get_features('geoportal:cobertura_vegetal')
df_veg.head()


## Carregando a camada de quadras viárias

In [ ]:
get_capabilities('viaria')

In [ ]:
df_quadras = get_features('geoportal:quadra_viaria_editada')
df_quadras.head()


## Carregando a camada de subprefeituras

In [ ]:
get_capabilities('subprefeitura')

In [ ]:
df_subs = get_features('geoportal:subprefeitura')
df_subs.head()

## Ajustando as projeções

Vamos revisar os sistemas de coordenadas de todos os geodataframes para garantir que estão na mesma projeção.

In [ ]:
for gdf in [df_geo, df_hid, df_censo, df_veg, df_quadras, df_subs]:
    print(gdf.columns[:5])
    print(gdf.crs)

Como o geodataframe do censo está em outro crs, precisamos convertê-lo para o `epsg:31983`.

In [13]:
df_censo = df_censo.to_crs('EPSG:31983')

In [ ]:
df_geo.explore()

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [18]:
output_dir = path.join('data', 'input', 'urbanismo')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, gdf in [('risco_hidrologico_original', df_hid),
               ('risco_geologico_original', df_geo),
               ('setores_censitarios_original', df_censo),
               ('cobertura_vegetal_original', df_veg),
               ('quadras_viarias_original', df_quadras),
               ('subprefeituras_original', df_subs)]:
    filename=path.join(output_dir, c)
    gdf.to_file(f'{filename}.gpkg', driver='GPKG', layer=c, overwrite=True)